In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl
import scipy.optimize

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
DATAPATH = os.path.join(PATH, 'datasets/data')


def linear_fit(x, a, b):
    
    return a + b * x



# load data

In [ ]:
os.listdir(os.path.join(RESULTPATH, 'scores'))

In [ ]:
os.listdir(os.path.join(DATAPATH))

In [ ]:
template_data = mf.data.MFDataset(os.path.join(DATAPATH, '211106_84_2cm_low_energy.h5'))

#beamforming_result = np.load(os.path.join(RESULTPATH,'211005_mf_84_100_template_ideal_scores_bottom_tri.npy'))

In [ ]:
template_metadata = pd.DataFrame(template_data.metadata)

In [ ]:
scores.argmax(axis=-1).shape

In [ ]:
scores = np.load(os.path.join(RESULTPATH, 'scores', '211106_84_25_2cm_low_energy_on_template.npy'))
ideal_scores = np.load(os.path.join(RESULTPATH, 'scores', '211112_84_25_self_scores.npy'))


match = scores.max(axis=-1) / ideal_scores[scores.argmax(axis=-1)]

# pick signal, sort scores with metadata, normalize

In [ ]:


energy_array = template_metadata['energy'].array
angle_array = template_metadata['theta_min'].array

unique_energy = np.sort(np.unique(template_metadata['energy'].array))
unique_angle = np.sort(np.unique(template_metadata['theta_min'].array))

energy_grid, angle_grid = np.meshgrid(unique_energy, unique_angle)

sorted_match = np.zeros(energy_grid.shape).flatten()

for i, pair in enumerate(zip(energy_grid.flatten(), angle_grid.flatten())):
    
    try:
        index = template_metadata[(template_metadata['energy'] == pair[0]) & (template_metadata['theta_min'] == pair[1])].index[0]
        sorted_match[i] = match[index]
    except:
        sorted_match[i] = 0.0
        
sorted_match = sorted_match.reshape(energy_grid.shape)

# plot

In [ ]:
sns.set_theme(style='ticks', context='talk')
cmap = sns.color_palette('mako', as_cmap=True)
fig = plt.figure(figsize=(13,8))

from matplotlib.colors import LogNorm

ax = fig.add_subplot(1,1,1)
img = ax.imshow(sorted_match, aspect='auto', interpolation='none', cmap=cmap, 
                norm=None, extent=(17574 - 18575, 18574 - 18575, 90, 84))
cbar = fig.colorbar(img)

ax.set_xlabel('Energy Below Window(eV)')
ax.set_ylabel('Pitch Angle (deg)')
name = f'211112_offaxis2cm_low_energy_3timeslice_match_ratio_25eV_window.png'
#ax.plot(signal_energy, signal_angle, 'r.')

plt.tight_layout()

#plt.savefig(os.path.join(PLOTPATH, name))

# plot qualitative

In [ ]:
sns.set_theme(style='ticks', context='talk')
cmap = sns.color_palette('mako', as_cmap=True)
fig = plt.figure(figsize=(13,8))

from matplotlib.colors import LogNorm

ax = fig.add_subplot(1,1,1)
img = ax.imshow(sorted_match, aspect='auto', interpolation='none', cmap='Paired', 
                norm=None, extent=(17574 - 18575, 18574 - 18575, 90, 84))
cbar = fig.colorbar(img)

ax.set_xlabel('Energy Below Window(eV)')
ax.set_ylabel('Pitch Angle (deg)')
name = f'211112_offaxis2cm_low_energy_3timeslice_match_ratio_25eV_window_qualitative.png'
#ax.plot(signal_energy, signal_angle, 'r.')

plt.tight_layout()

#plt.savefig(os.path.join(PLOTPATH, name))

In [ ]:
x

sorted_match[np.argwhere(sorted_match>0.8)[]].shape